## 1. 仅保存模型架构：

仅保存架构，把权重值和优化器配置等都丢掉。—— 保存的文件格式：

In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
# 加载数据集:
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.fashion_mnist.load_data()

# 归一化：不喜欢太大的数！
train_image = train_image / 255
test_image = test_image / 255

In [3]:
model = tf.keras.Sequential( [
    tf.keras.layers.Flatten( input_shape = (28,28) ),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(10, activation = 'softmax')
] )

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',  
    metrics = ['acc'] 
)

model.fit(train_image, train_label, epochs = 5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 55us/sample - loss: 0.4984 - acc: 0.8242
Epoch 2/5
60000/60000 [==============================] - 3s 47us/sample - loss: 0.3734 - acc: 0.8648
Epoch 3/5
60000/60000 [==============================] - 3s 45us/sample - loss: 0.3369 - acc: 0.8779
Epoch 4/5
60000/60000 [==============================] - 3s 44us/sample - loss: 0.3139 - acc: 0.8854
Epoch 5/5
60000/60000 [==============================] - 3s 45us/sample - loss: 0.2950 - acc: 0.8904


### 仅保存模型框架：

In [6]:
json_model = model.to_json()  

In [7]:
json_model  # 想写到磁盘里，就用python最简单的write、open那些函数写或读即可。

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "Flatten", "config": {"name": "flatten", "trainable": true, "batch_input_shape": [null, 28, 28], "dtype": "float32", "data_format": "channels_last"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": "float32", "units": 128, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": 

In [8]:
# 如何使用：
reinitialized_model = tf.keras.models.model_from_json( json_model )

In [9]:
reinitialized_model.evaluate( test_image, test_label, verbose = 0 )  # 报RuntimeError错误，说明是编译器没配置好！

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [10]:
reinitialized_model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',  
    metrics = ['acc'] 
)

In [12]:
reinitialized_model.evaluate( test_image, test_label, verbose = 0 )  # 权重都没训练，用的都是随机值！

[2.41361704788208, 0.082]

## 2. 仅保存模型训练好的权重：

用处：程序不会丢，把已经训练不错的权重值保存起来，下次用程序中的模型继承该权重即可继续训练。

注意：
- 当把权重值写入磁盘后，虽然都是保存成了.h5格式，但是导入不能用tf.keras.models.load_model！—— 这是专门导入完整模型的函数！
- 导入磁盘.h5文件时，用的是load_weights；直接使用绑定权重的变量，用的是set_weights！

In [13]:
weights = model.get_weights()

In [16]:
len(weights)  # 带上输入层、一共有4层！

4

In [17]:
weights[0].shape  # 每个元素是一个array！记录对应的权重值！

(784, 128)

In [19]:
# 如何使用：
reinitialized_model.set_weights( weights )

In [21]:
reinitialized_model.evaluate( test_image, test_label, verbose = 0 )   # 说明是训练好的权重！

[0.3595873064398766, 0.8687]

In [23]:
# 写到磁盘上：因为每个元素是array，不适合用普通的write和open函数；所以keras有专门的高阶保存函数！
model.save_weights( '10.1_weights.h5' )

In [26]:
reinitialized_model.load_weights( '10.1_weights.h5' )  # 括号内是权重文件地址

In [27]:
reinitialized_model.evaluate( test_image, test_label, verbose = 0 )

[0.3595873064398766, 0.8687]

## 3. 总结：

仅保存框架 + 仅保存权重 ≠ 模型的全部保存

因为模型的全部保存，还多保存了优化器的配置！—— 上面例子可看出：新的model还需model.compile()！

为什么优化器也会在训练中发生变化：比如使用Adam优化器，这是一种自适应的优化器，它里面的很多参数都随着训练会自适应调整。